In [8]:
import pandas as pd
TrainingLabels = pd.read_csv('Training labels.csv', index_col='id')
TrainingFeatures = pd.read_csv('Training features.csv', index_col='id')
TestFeatures = pd.read_csv('Test features.csv', index_col='id')
TrainingFeatures.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [18]:
preprocess = pd.concat([TrainingFeatures,TestFeatures])
preprocess.drop(['wpt_name','num_private','subvillage','region','lga','ward','recorded_by','scheme_name',
'extraction_type_class','extraction_type','payment_type','water_quality','quantity_group','source_type',
'source','waterpoint_type','date_recorded', 'longitude','latitude','management_group'],axis=1,inplace=True)

In [19]:
import numpy as nan
preprocess['funder'] = preprocess['funder'].str.lower()
preprocess['funder'].fillna(preprocess.installer, inplace=True)
preprocess['funder'].fillna('other', inplace=True)
funder_reduction = ['kkkt','vision','foundat','tlc','wsd','norad','sema','wfp','ded','unicef','danida','hes','rws','dhv','isf']

for category in funder_reduction:
    subCategory = preprocess[preprocess['funder'].str.contains(category)]
    unique = subCategory['funder'].unique()
    preprocess['funder'].replace(to_replace=unique,value=category, inplace=True)

multi = [
[['hifab','amref','oxfa','wateraid','Wate Aid','Water Aid','adra','plan','Ces(gmbh)','Ces (gmbh)','shipo','dmdd','oikos'],'project'],
[['mzee','priva','mwita','ltd','bank','kanis'],'private'],
[['municipal council', 'distric','govern','ministry','supply','adb','rudep'],'govern'],
[['rc','roman',' mission','cathoric','st ','churc','anglican','tcrs'],'church'],
[['total land care','total landcare','totaland care','totoland','totoland care'],'totallandcare'],
[['tassaf','tasaf'],'tasaf'],
[['ngo ',' ngo'],'ngo'],
[['rotary','club'],'club'],
[['islam','muslim'],'muslim'],
[['said ',' said'],'said'],
[['british','japan','egypt','belgian','italy','iran','china','finland','irish','swedish','german','netherlan','european','kuwait','african','jaica','jica'],'foreign'],
[['village','community','rural','halmashauri','lga'],'village'],
[['lga','adb'],'lga & adb'],
[['Fin Water','Fini Water','Finn Water','Finw','Finwater','Fiwater'],'fini']
]
for category in multi:
    for name in category[0]:
        subCategory = preprocess.loc[preprocess['funder'].str.contains(name)]
        unique = subCategory['funder'].unique()
        preprocess['funder'].replace(to_replace=unique,value=category[1], inplace=True)

preprocess['funder'].replace(to_replace=('mission','missio','missi'),value='church', inplace=True)

preprocess['funder'].replace(to_replace=('0', 'nan', '-', ''), value='other', inplace=True)
count = preprocess.groupby('funder')['funder'].count()
other_list = count[count<500].index.tolist()
preprocess['funder'].replace(to_replace=other_list, value='other', inplace=True)  

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(preprocess.groupby('funder')['funder'].count().sort_values(ascending=False))

C:\Users\Yasith\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


funder
other      24819
govern     15431
danida      3921
project     3661
private     3402
foreign     3163
church      3077
hes         2802
kkkt        1953
rws         1742
vision      1603
village     1430
unicef      1350
tasaf       1316
dhv         1146
norad       1078
dwsp        1012
water        739
dwe          605
Name: funder, dtype: int64


In [20]:
import numpy as np

preprocess['gps_height'] = preprocess['gps_height'].abs()

preprocess['age'] = 2013 - preprocess.construction_year
preprocess.drop('construction_year', axis = 1, inplace = True)
preprocess.drop('installer', axis = 1, inplace = True)

pop_bins = [-1,0,50, 200, 31000]
pop_labels = [1,2,3,4]
TrainingFeatures['pop_binned'] = pd.cut(TrainingFeatures['population'], bins=pop_bins, labels=pop_labels)
preprocess.drop('population', axis = 1, inplace = True)


bool_mapper = {"False":0, "True":1}
preprocess["permit"] = preprocess["permit"].replace(bool_mapper)
preprocess["public_meeting"] = preprocess["public_meeting"].replace(bool_mapper)

In [21]:
object_columns = preprocess.select_dtypes(include=['object']).columns
for object_col in object_columns:
    preprocess[object_col] = preprocess[object_col].astype('category')

In [22]:
for bool_col in ['permit','public_meeting']:
    preprocess[bool_col] = preprocess[bool_col].astype('boolean')

In [23]:
preprocess.dtypes

amount_tsh                float64
funder                   category
gps_height                  int64
basin                    category
region_code                 int64
district_code               int64
public_meeting            boolean
scheme_management        category
permit                    boolean
extraction_type_group    category
management               category
payment                  category
quality_group            category
quantity                 category
source_class             category
waterpoint_type_group    category
age                         int64
dtype: object

In [24]:
preprocess.isna().sum()

amount_tsh                  0
funder                      0
gps_height                  0
basin                       0
region_code                 0
district_code               0
public_meeting           4155
scheme_management        4846
permit                   3793
extraction_type_group       0
management                  0
payment                     0
quality_group               0
quantity                    0
source_class                0
waterpoint_type_group       0
age                         0
dtype: int64

In [25]:
preprocess.nunique

<bound method DataFrame.nunique of        amount_tsh   funder  gps_height                    basin  region_code  \
id                                                                             
69572      6000.0   church        1390               Lake Nyasa           11   
8776          0.0    other        1399            Lake Victoria           20   
34310        25.0    other         686                  Pangani           21   
67743         0.0   unicef         263  Ruvuma / Southern Coast           90   
19728         0.0    other           0            Lake Victoria           18   
...           ...      ...         ...                      ...          ...   
39307         0.0   danida          34              Wami / Ruvu            6   
18990      1000.0    other           0                  Pangani            4   
28749         0.0    other        1476                 Internal           13   
33492         0.0  foreign         998               Lake Nyasa           10   
68707

In [28]:
preprocess['age'].replace(to_replace=2013, value=np.nan, inplace=True)  
mean_fill = ['age']
for fill in mean_fill:
    preprocess[fill].fillna(preprocess[fill].mean(), inplace=True)
mode_fill = ['public_meeting','permit']
for fill in mode_fill:
    preprocess[fill].fillna(preprocess[fill].mode()[0], inplace=True)

In [29]:
TrainingFeatures = preprocess.iloc[:59400,:]
TestFeatures = preprocess.iloc[59400:,:]
TrainingFeatures['status'] = TrainingLabels['status_group']
TrainingFeatures.to_csv('training_cleaned_min.csv', index=True)
TestFeatures.to_csv('test_cleaned_min.csv', index=True)

C:\Users\Yasith\AppData\Local\Temp/ipykernel_6684/698745974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TrainingFeatures['status'] = TrainingLabels['status_group']


In [ ]:
TestFeatures.dtypes

amount_tsh                float64
funder                   category
gps_height                  int64
longitude                 float64
latitude                  float64
basin                    category
subvillage               category
region                   category
region_code                 int64
district_code               int64
lga                      category
ward                     category
public_meeting            boolean
scheme_management        category
scheme_name              category
permit                    boolean
extraction_type          category
extraction_type_group    category
extraction_type_class    category
management               category
management_group         category
payment                  category
payment_type             category
water_quality            category
quality_group            category
quantity                 category
quantity_group           category
source                   category
source_type              category
source_class  

In [3]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

train = h2o.import_file("training_cleaned_min.csv")
test = h2o.import_file("test_cleaned_min.csv")

x = train.columns
y = "status"
x.remove(y)

aml = H2OAutoML()
aml.train(x=x, y=y, training_frame=train)
# from  autosklearn.classification import AutoSklearnClassifier
# cat = AutoSklearnClassifier()
# cat.fit(TrainingFeatures,TrainingLabels)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 15+36-1562, mixed mode, sharing)
  Starting server from C:\Users\Yasith\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Yasith\AppData\Local\Temp\tmpvk1_088l
  JVM stdout: C:\Users\Yasith\AppData\Local\Temp\tmpvk1_088l\h2o_Yasith_started_from_python.out
  JVM stderr: C:\Users\Yasith\AppData\Local\Temp\tmpvk1_088l\h2o_Yasith_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_Yasith_h67lo9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.982 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
20:00:55.450: AutoML: XGBoost is not available; skipping it.

████
20:01:19.289: GLM_1_AutoML_20210915_200055 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: Index 102 out of bounds for length 102

████████████████████████████████████████████████████| 100%


In [7]:
aml.get_best_model().params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_grid__1_AutoML_20210915_200055_model_35',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_grid__1_AutoML_20210915_200055_model_35'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_training_cleaned_min.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_training_cleaned_min.hex'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_training_cleaned_min.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_training_cleaned_min.hex'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'actual': 5, 'input': 5},
 'keep_cross_v

In [5]:
#s = cat.predict(TestFeatures)
#s = le.inverse_transform(s)

preds = aml.predict(test)
pred_pd = preds.as_data_frame(use_pandas=True)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [15]:
pred_pd['id'] = TestFeatures.index
pred_pd.rename(columns={'predict':'status_group'},inplace=True)
pred_pd = pred_pd[['id','status_group']]
pred_pd.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [16]:
import pandas as pd

pred_pd.to_csv('output_yasith_1.csv', index=False)

In [32]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")

KeyboardInterrupt: 

In [21]:
cat.sprint_statistics()

'auto-sklearn results:\n  Dataset name: ece19e46-1609-11ec-800f-0242ac110002\n  Metric: accuracy\n  Best validation score: 0.804204\n  Number of target algorithm runs: 73\n  Number of successful target algorithm runs: 45\n  Number of crashed target algorithm runs: 2\n  Number of target algorithms that exceeded the time limit: 4\n  Number of target algorithms that exceeded the memory limit: 22\n'